In [1]:
# Echo Shepherd — an original Pygame mini-game
# Author: ChatGPT (GPT-5 Thinking)
# Requirements: pygame (pip install pygame)

import math
import random
import sys
from collections import deque

import pygame

# --------------------------- Config ---------------------------
WIDTH, HEIGHT = 960, 600
FPS = 60

PLAYER_RADIUS = 12
PLAYER_SPEED = 240  # pixels per second

ECHO_DELAY_SEC_START = 2.0      # delay between player path and echo path (will shrink each level)
ECHO_SPEED_FACTOR = 1.0         # echo follows at same speed as player path playback
ECHO_COLLISION_PENALTY = 1      # lives lost on echo touch
MAX_LIVES = 3

NOTE_COUNT = 3
NOTE_RADIUS = 10
NOTE_SPEED_BASE = 90            # base drift speed (per second)
NOTE_SPEED_LEVEL_ADD = 18       # extra per level
NOTE_SPOOK_DISTANCE = 40        # if player closer than this, note starts fleeing
NOTE_SPOOK_FORCE = 260          # fleeing impulse
NOTE_FRICTION = 0.92

GOAL_RADIUS = 30
CAPTURE_DWELL_SEC = 1.0         # how long the correct note must stay in goal

LEVEL_TIME_SEC = 60

BG_COLOR = (18, 20, 26)
WALL_COLOR = (40, 44, 52)
TEXT_COLOR = (230, 232, 238)
DIM_TEXT = (160, 166, 178)
PLAYER_COLOR = (50, 200, 255)
ECHO_COLOR = (255, 120, 120)
GOAL_RING = (255, 255, 255)

NOTE_COLORS = [
    (255, 199, 95),   # amber
    (140, 220, 120),  # green
    (130, 170, 255),  # blue
    (255, 130, 170),  # pink
    (200, 160, 255),  # violet
]

# Deterministic-ish randomness toggle if you want reproducibility
RANDOM_SEED = None  # set to an int to get the same layouts every time


# --------------------------- Helpers ---------------------------
def clamp(x, a, b):
    return max(a, min(b, x))


def length(vx, vy):
    return math.hypot(vx, vy)


def normalize(vx, vy):
    d = math.hypot(vx, vy)
    if d == 0:
        return 0.0, 0.0
    return vx / d, vy / d


def circle_collide(x1, y1, r1, x2, y2, r2):
    return (x1 - x2) ** 2 + (y1 - y2) ** 2 <= (r1 + r2) ** 2


def wrap_inside_walls(x, y, r, walls):
    # keep inside screen bounds and away from walls
    x = clamp(x, r, WIDTH - r)
    y = clamp(y, r, HEIGHT - r)
    # push out of walls if overlapping
    for rx, ry, rw, rh in walls:
        nearest_x = clamp(x, rx, rx + rw)
        nearest_y = clamp(y, ry, ry + rh)
        dx, dy = x - nearest_x, y - nearest_y
        dist2 = dx * dx + dy * dy
        if dist2 < r * r:
            d = math.sqrt(max(dist2, 1e-8))
            nx, ny = (dx / d, dy / d) if d != 0 else (1, 0)
            # move just outside the wall
            x = nearest_x + nx * (r + 0.5)
            y = nearest_y + ny * (r + 0.5)
    return x, y


# --------------------------- Entities ---------------------------
class Player:
    def __init__(self, x, y):
        self.x, self.y = x, y
        self.color = PLAYER_COLOR

    def update(self, dt, keys, walls):
        dx = (keys[pygame.K_d] or keys[pygame.K_RIGHT]) - (keys[pygame.K_a] or keys[pygame.K_LEFT])
        dy = (keys[pygame.K_s] or keys[pygame.K_DOWN]) - (keys[pygame.K_w] or keys[pygame.K_UP])

        # boolean -> int conversion yields True=1, but above is bools; fix with int()
        dx = int(dx)
        dy = int(dy)

        if dx and dy:
            # normalize diagonal to keep speed consistent
            nx, ny = normalize(dx, dy)
            dx, dy = nx, ny
        speed = PLAYER_SPEED * dt
        self.x += dx * speed
        self.y += dy * speed
        self.x, self.y = wrap_inside_walls(self.x, self.y, PLAYER_RADIUS, walls)

    def draw(self, surf):
        pygame.draw.circle(surf, self.color, (int(self.x), int(self.y)), PLAYER_RADIUS)
        pygame.draw.circle(surf, (255, 255, 255), (int(self.x), int(self.y)), PLAYER_RADIUS, 1)


class Echo:
    def __init__(self, delay_sec):
        self.delay_sec = delay_sec
        self.history = deque()
        self.pos = None

    def reset(self, delay_sec):
        self.delay_sec = delay_sec
        self.history.clear()
        self.pos = None

    def push(self, x, y):
        self.history.append((x, y))

    def update(self, dt, fps):
        # Maintain delay by keeping only recent positions
        maxlen = max(1, int(self.delay_sec * fps))
        while len(self.history) > maxlen:
            self.pos = self.history.popleft()

    def draw(self, surf):
        if self.pos:
            pygame.draw.circle(surf, ECHO_COLOR, (int(self.pos[0]), int(self.pos[1])), PLAYER_RADIUS)
            pygame.draw.circle(surf, (255, 255, 255), (int(self.pos[0]), int(self.pos[1])), PLAYER_RADIUS, 1)


class Note:
    def __init__(self, x, y, color, speed):
        self.x, self.y = x, y
        self.vx = random.uniform(-1, 1) * speed
        self.vy = random.uniform(-1, 1) * speed
        self.color = color
        self.base_speed = speed
        self.in_goal_time = 0.0

    def update(self, dt, player, walls):
        # random wander
        jitter = 40.0
        self.vx += random.uniform(-jitter, jitter) * dt
        self.vy += random.uniform(-jitter, jitter) * dt

        # spook if player too close — flee away
        if length(player.x - self.x, player.y - self.y) < NOTE_SPOOK_DISTANCE:
            nx, ny = normalize(self.x - player.x, self.y - player.y)
            self.vx += nx * NOTE_SPOOK_FORCE * dt
            self.vy += ny * NOTE_SPOOK_FORCE * dt

        # friction to keep velocities bounded
        self.vx *= NOTE_FRICTION
        self.vy *= NOTE_FRICTION

        # clamp to reasonable speed
        v = length(self.vx, self.vy)
        max_v = self.base_speed * 1.7
        if v > max_v:
            nx, ny = self.vx / v, self.vy / v
            self.vx, self.vy = nx * max_v, ny * max_v

        self.x += self.vx * dt
        self.y += self.vy * dt

        # bounce on walls & screen bounds
        # screen edges
        if self.x < NOTE_RADIUS:
            self.x, self.vx = NOTE_RADIUS, abs(self.vx)
        elif self.x > WIDTH - NOTE_RADIUS:
            self.x, self.vx = WIDTH - NOTE_RADIUS, -abs(self.vx)
        if self.y < NOTE_RADIUS:
            self.y, self.vy = NOTE_RADIUS, abs(self.vy)
        elif self.y > HEIGHT - NOTE_RADIUS:
            self.y, self.vy = HEIGHT - NOTE_RADIUS, -abs(self.vy)

        # simple wall collisions (AABB reflect)
        for rx, ry, rw, rh in walls:
            if rx < self.x < rx + rw and ry - NOTE_RADIUS < self.y < ry + rh + NOTE_RADIUS:
                if ry <= self.y <= ry + rh:
                    # horizontal bounce on vertical sides
                    if self.x < rx + rw / 2:
                        self.x = rx - NOTE_RADIUS
                        self.vx = -abs(self.vx)
                    else:
                        self.x = rx + rw + NOTE_RADIUS
                        self.vx = abs(self.vx)
            if ry < self.y < ry + rh and rx - NOTE_RADIUS < self.x < rx + rw + NOTE_RADIUS:
                if rx <= self.x <= rx + rw:
                    # vertical bounce on horizontal sides
                    if self.y < ry + rh / 2:
                        self.y = ry - NOTE_RADIUS
                        self.vy = -abs(self.vy)
                    else:
                        self.y = ry + rh + NOTE_RADIUS
                        self.vy = abs(self.vy)

    def draw(self, surf):
        pygame.draw.circle(surf, self.color, (int(self.x), int(self.y)), NOTE_RADIUS)
        pygame.draw.circle(surf, (20, 20, 20), (int(self.x), int(self.y)), NOTE_RADIUS, 1)


class Goal:
    def __init__(self, x, y, order_colors):
        self.x, self.y = x, y
        self.order = list(order_colors)  # queue of colors to accept
        self.capture_timer = 0.0

    def reset(self, order_colors):
        self.order = list(order_colors)
        self.capture_timer = 0.0

    def draw(self, surf):
        # outer ring
        pygame.draw.circle(surf, GOAL_RING, (int(self.x), int(self.y)), GOAL_RADIUS, 2)
        # show required color
        if self.order:
            req = self.order[0]
            pygame.draw.circle(surf, req, (int(self.x), int(self.y)), GOAL_RADIUS - 6, 2)
        else:
            # completed — draw a star-ish finish
            pygame.draw.circle(surf, (255, 255, 255), (int(self.x), int(self.y)), GOAL_RADIUS - 6, 1)

    def update_capture(self, dt, notes):
        if not self.order:
            return False  # already done
        required = self.order[0]
        captured = None
        for n in notes:
            if n.color == required and circle_collide(self.x, self.y, GOAL_RADIUS - 3, n.x, n.y, NOTE_RADIUS):
                n.in_goal_time += dt
                if n.in_goal_time >= CAPTURE_DWELL_SEC:
                    captured = n
                # decay other notes' timers
            else:
                n.in_goal_time = max(0.0, n.in_goal_time - dt * 0.5)

        if captured:
            # remove that color from order, respawn the note elsewhere
            self.order.pop(0)
            captured.in_goal_time = 0.0
            captured.x = random.randint(80, WIDTH - 80)
            captured.y = random.randint(80, HEIGHT - 80)
            captured.vx = random.uniform(-1, 1) * captured.base_speed
            captured.vy = random.uniform(-1, 1) * captured.base_speed
            return True
        return False


# --------------------------- World/Level ---------------------------
def make_walls():
    # A few soft maze walls
    rng = random.Random(random.randint(0, 99999))
    walls = []
    for _ in range(5):
        w = rng.randint(120, 200)
        h = rng.randint(24, 36)
        x = rng.randint(60, WIDTH - w - 60)
        y = rng.randint(60, HEIGHT - h - 60)
        walls.append((x, y, w, h))
    for _ in range(5):
        w = rng.randint(24, 36)
        h = rng.randint(120, 200)
        x = rng.randint(60, WIDTH - w - 60)
        y = rng.randint(60, HEIGHT - h - 60)
        walls.append((x, y, w, h))
    return walls


def draw_walls(surf, walls):
    for x, y, w, h in walls:
        pygame.draw.rect(surf, WALL_COLOR, (x, y, w, h), border_radius=8)


# --------------------------- Game Loop ---------------------------
class Game:
    def __init__(self):
        pygame.init()
        pygame.display.set_caption("Echo Shepherd")
        self.screen = pygame.display.set_mode((WIDTH, HEIGHT))
        self.clock = pygame.time.Clock()
        self.font = pygame.font.SysFont("consolas", 20)
        self.big = pygame.font.SysFont("consolas", 36)
        self.running = True
        self.paused = False

        if RANDOM_SEED is not None:
            random.seed(RANDOM_SEED)

        self.level = 1
        self.reset_level(hard_reset=True)

    def reset_level(self, hard_reset=False):
        if hard_reset:
            self.level = 1
        self.lives = MAX_LIVES
        self.time_left = LEVEL_TIME_SEC
        self.walls = make_walls()

        px, py = WIDTH * 0.15, HEIGHT * 0.5
        self.player = Player(px, py)
        delay = max(0.8, ECHO_DELAY_SEC_START - (self.level - 1) * 0.3)
        self.echo = Echo(delay)

        # Goal and notes setup
        order_colors = random.sample(NOTE_COLORS, NOTE_COUNT)
        self.goal = Goal(int(WIDTH * 0.85), int(HEIGHT * 0.5), order_colors)
        speed = NOTE_SPEED_BASE + (self.level - 1) * NOTE_SPEED_LEVEL_ADD
        self.notes = []
        for i in range(NOTE_COUNT):
            # spawn away from the goal
            x = random.randint(80, int(WIDTH * 0.55))
            y = random.randint(80, HEIGHT - 80)
            self.notes.append(Note(x, y, order_colors[i], speed))

        # Clear echo history, pre-fill with current pl


pygame 2.6.1 (SDL 2.28.4, Python 3.12.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Echo Shepherd — now with multi-monitor support
# Author: ChatGPT (GPT-5)
# Requirements: pygame (pip install pygame)

import math
import random
import sys
from collections import deque
import pygame
import os

# --------------------------- Config ---------------------------
WIDTH, HEIGHT = 960, 600
FPS = 60

PLAYER_RADIUS = 12
PLAYER_SPEED = 240  # pixels per second

ECHO_DELAY_SEC_START = 2.0
ECHO_SPEED_FACTOR = 1.0
ECHO_COLLISION_PENALTY = 1
MAX_LIVES = 3

NOTE_COUNT = 3
NOTE_RADIUS = 10
NOTE_SPEED_BASE = 90
NOTE_SPEED_LEVEL_ADD = 18
NOTE_SPOOK_DISTANCE = 40
NOTE_SPOOK_FORCE = 260
NOTE_FRICTION = 0.92

GOAL_RADIUS = 30
CAPTURE_DWELL_SEC = 1.0

LEVEL_TIME_SEC = 60

BG_COLOR = (18, 20, 26)
WALL_COLOR = (40, 44, 52)
TEXT_COLOR = (230, 232, 238)
DIM_TEXT = (160, 166, 178)
PLAYER_COLOR = (50, 200, 255)
ECHO_COLOR = (255, 120, 120)
GOAL_RING = (255, 255, 255)

NOTE_COLORS = [
    (255, 199, 95),
    (140, 220, 120),
    (130, 170, 255),
    (255, 130, 170),
    (200, 160, 255),
]

RANDOM_SEED = None  # for reproducibility if needed
FORCE_FULLSCREEN = False  # set True for fullscreen on 2nd monitor


# --------------------------- Helpers ---------------------------
def clamp(x, a, b):
    return max(a, min(b, x))


def length(vx, vy):
    return math.hypot(vx, vy)


def normalize(vx, vy):
    d = math.hypot(vx, vy)
    if d == 0:
        return 0.0, 0.0
    return vx / d, vy / d


def circle_collide(x1, y1, r1, x2, y2, r2):
    return (x1 - x2) ** 2 + (y1 - y2) ** 2 <= (r1 + r2) ** 2


def wrap_inside_walls(x, y, r, walls):
    x = clamp(x, r, WIDTH - r)
    y = clamp(y, r, HEIGHT - r)
    for rx, ry, rw, rh in walls:
        nearest_x = clamp(x, rx, rx + rw)
        nearest_y = clamp(y, ry, ry + rh)
        dx, dy = x - nearest_x, y - nearest_y
        dist2 = dx * dx + dy * dy
        if dist2 < r * r:
            d = math.sqrt(max(dist2, 1e-8))
            nx, ny = (dx / d, dy / d) if d != 0 else (1, 0)
            x = nearest_x + nx * (r + 0.5)
            y = nearest_y + ny * (r + 0.5)
    return x, y


# --------------------------- Entities ---------------------------
class Player:
    def __init__(self, x, y):
        self.x, self.y = x, y
        self.color = PLAYER_COLOR

    def update(self, dt, keys, walls):
        dx = (keys[pygame.K_d] or keys[pygame.K_RIGHT]) - (keys[pygame.K_a] or keys[pygame.K_LEFT])
        dy = (keys[pygame.K_s] or keys[pygame.K_DOWN]) - (keys[pygame.K_w] or keys[pygame.K_UP])
        dx, dy = int(dx), int(dy)

        if dx and dy:
            nx, ny = normalize(dx, dy)
            dx, dy = nx, ny
        speed = PLAYER_SPEED * dt
        self.x += dx * speed
        self.y += dy * speed
        self.x, self.y = wrap_inside_walls(self.x, self.y, PLAYER_RADIUS, walls)

    def draw(self, surf):
        pygame.draw.circle(surf, self.color, (int(self.x), int(self.y)), PLAYER_RADIUS)
        pygame.draw.circle(surf, (255, 255, 255), (int(self.x), int(self.y)), PLAYER_RADIUS, 1)


class Echo:
    def __init__(self, delay_sec):
        self.delay_sec = delay_sec
        self.history = deque()
        self.pos = None

    def reset(self, delay_sec):
        self.delay_sec = delay_sec
        self.history.clear()
        self.pos = None

    def push(self, x, y):
        self.history.append((x, y))

    def update(self, dt, fps):
        maxlen = max(1, int(self.delay_sec * fps))
        while len(self.history) > maxlen:
            self.pos = self.history.popleft()

    def draw(self, surf):
        if self.pos:
            pygame.draw.circle(surf, ECHO_COLOR, (int(self.pos[0]), int(self.pos[1])), PLAYER_RADIUS)
            pygame.draw.circle(surf, (255, 255, 255), (int(self.pos[0]), int(self.pos[1])), PLAYER_RADIUS, 1)


class Note:
    def __init__(self, x, y, color, speed):
        self.x, self.y = x, y
        self.vx = random.uniform(-1, 1) * speed
        self.vy = random.uniform(-1, 1) * speed
        self.color = color
        self.base_speed = speed
        self.in_goal_time = 0.0

    def update(self, dt, player, walls):
        jitter = 40.0
        self.vx += random.uniform(-jitter, jitter) * dt
        self.vy += random.uniform(-jitter, jitter) * dt

        if length(player.x - self.x, player.y - self.y) < NOTE_SPOOK_DISTANCE:
            nx, ny = normalize(self.x - player.x, self.y - player.y)
            self.vx += nx * NOTE_SPOOK_FORCE * dt
            self.vy += ny * NOTE_SPOOK_FORCE * dt

        self.vx *= NOTE_FRICTION
        self.vy *= NOTE_FRICTION

        v = length(self.vx, self.vy)
        max_v = self.base_speed * 1.7
        if v > max_v:
            nx, ny = self.vx / v, self.vy / v
            self.vx, self.vy = nx * max_v, ny * max_v

        self.x += self.vx * dt
        self.y += self.vy * dt

        if self.x < NOTE_RADIUS:
            self.x, self.vx = NOTE_RADIUS, abs(self.vx)
        elif self.x > WIDTH - NOTE_RADIUS:
            self.x, self.vx = WIDTH - NOTE_RADIUS, -abs(self.vx)
        if self.y < NOTE_RADIUS:
            self.y, self.vy = NOTE_RADIUS, abs(self.vy)
        elif self.y > HEIGHT - NOTE_RADIUS:
            self.y, self.vy = HEIGHT - NOTE_RADIUS, -abs(self.vy)

    def draw(self, surf):
        pygame.draw.circle(surf, self.color, (int(self.x), int(self.y)), NOTE_RADIUS)
        pygame.draw.circle(surf, (20, 20, 20), (int(self.x), int(self.y)), NOTE_RADIUS, 1)


class Goal:
    def __init__(self, x, y, order_colors):
        self.x, self.y = x, y
        self.order = list(order_colors)
        self.capture_timer = 0.0

    def reset(self, order_colors):
        self.order = list(order_colors)
        self.capture_timer = 0.0

    def draw(self, surf):
        pygame.draw.circle(surf, GOAL_RING, (int(self.x), int(self.y)), GOAL_RADIUS, 2)
        if self.order:
            req = self.order[0]
            pygame.draw.circle(surf, req, (int(self.x), int(self.y)), GOAL_RADIUS - 6, 2)
        else:
            pygame.draw.circle(surf, (255, 255, 255), (int(self.x), int(self.y)), GOAL_RADIUS - 6, 1)

    def update_capture(self, dt, notes):
        if not self.order:
            return False
        required = self.order[0]
        captured = None
        for n in notes:
            if n.color == required and circle_collide(self.x, self.y, GOAL_RADIUS - 3, n.x, n.y, NOTE_RADIUS):
                n.in_goal_time += dt
                if n.in_goal_time >= CAPTURE_DWELL_SEC:
                    captured = n
            else:
                n.in_goal_time = max(0.0, n.in_goal_time - dt * 0.5)

        if captured:
            self.order.pop(0)
            captured.in_goal_time = 0.0
            captured.x = random.randint(80, WIDTH - 80)
            captured.y = random.randint(80, HEIGHT - 80)
            return True
        return False


# --------------------------- Game Loop ---------------------------
class Game:
    def __init__(self):
        pygame.init()
        pygame.display.set_caption("Echo Shepherd")

        # ---------------- Monitor selection ----------------
        displays = pygame.display.get_num_video_displays()
        print("Detected displays:", displays)

        if displays > 1:
            display_index = 1  # use second screen
            if FORCE_FULLSCREEN:
                info = pygame.display.get_desktop_sizes()[display_index]
                self.screen = pygame.display.set_mode(info, pygame.FULLSCREEN, display=display_index)
            else:
                self.screen = pygame.display.set_mode((WIDTH, HEIGHT), pygame.SHOWN, display=display_index)
        else:
            self.screen = pygame.display.set_mode((WIDTH, HEIGHT))

        # ---------------------------------------------------
        self.clock = pygame.time.Clock()
        self.font = pygame.font.SysFont("consolas", 20)
        self.big = pygame.font.SysFont("consolas", 36)
        self.running = True
        self.paused = False

        if RANDOM_SEED is not None:
            random.seed(RANDOM_SEED)

        self.level = 1
        self.reset_level(hard_reset=True)

    # rest of game code from previous version...
    # (update, draw_hud, run, etc. — unchanged)
    # --------------------------- (Keep the same as before) ---------------------------


if __name__ == "__main__":
    Game().run()


AttributeError: module 'pygame.display' has no attribute 'get_num_video_displays'

In [3]:
import tkinter as tk
import random
import time
import threading

class ColorMemoryGame:
    def __init__(self, root):
        self.root = root
        self.root.title("🎮 Color Memory Tiles")
        self.root.config(bg="black")

        # Game variables
        self.tiles = {}
        self.colors = ["red", "green", "blue", "yellow"]
        self.sequence = []
        self.user_sequence = []
        self.level = 0
        self.locked = True  # prevents user input while showing sequence

        # Score label
        self.score_label = tk.Label(root, text="Score: 0", font=("Arial", 20, "bold"), bg="black", fg="white")
        self.score_label.pack(pady=20)

        # Tile grid
        self.grid_frame = tk.Frame(root, bg="black")
        self.grid_frame.pack()

        # Create 2x2 grid of color tiles
        for i, color in enumerate(self.colors):
            btn = tk.Button(self.grid_frame, bg=color, width=12, height=6,
                            command=lambda c=color: self.tile_click(c))
            btn.grid(row=i//2, column=i%2, padx=10, pady=10)
            self.tiles[color] = btn

        # Start button
        self.start_btn = tk.Button(root, text="▶ Start Game", font=("Arial", 16, "bold"),
                                   bg="white", fg="black", command=self.start_game)
        self.start_btn.pack(pady=20)

        # Status message
        self.message = tk.Label(root, text="", font=("Arial", 16), bg="black", fg="white")
        self.message.pack(pady=10)

    def start_game(self):
        self.level = 0
        self.sequence = []
        self.user_sequence = []
        self.update_score()
        self.message.config(text="")
        self.next_round()

    def next_round(self):
        self.level += 1
        self.user_sequence = []
        new_color = random.choice(self.colors)
        self.sequence.append(new_color)
        self.message.config(text=f"Round {self.level} - Watch carefully!")
        self.root.after(1000, self.show_sequence)

    def show_sequence(self):
        self.locked = True
        delay = 1000
        for i, color in enumerate(self.sequence):
            self.root.after(i * delay, lambda c=color: self.flash_tile(c))
        self.root.after(len(self.sequence) * delay, self.unlock_input)

    def flash_tile(self, color):
        btn = self.tiles[color]
        original = btn.cget("bg")
        btn.config(bg="white")
        self.root.update()
        self.root.after(400, lambda: btn.config(bg=color))

    def unlock_input(self):
        self.message.config(text="Now repeat the sequence!")
        self.locked = False

    def tile_click(self, color):
        if self.locked:
            return
        self.user_sequence.append(color)
        self.flash_tile(color)

        # Check progress
        if self.user_sequence[-1] != self.sequence[len(self.user_sequence) - 1]:
            self.game_over()
            return

        # Round complete
        if len(self.user_sequence) == len(self.sequence):
            self.update_score()
            self.message.config(text="✅ Correct! Get ready for next round...")
            self.root.after(1500, self.next_round)

    def update_score(self):
        self.score_label.config(text=f"Score: {self.level}")

    def game_over(self):
        self.message.config(text="❌ Wrong! Game Over!")
        self.locked = True
        self.start_btn.config(text="▶ Restart Game")

# Run the game
if __name__ == "__main__":
    root = tk.Tk()
    game = ColorMemoryGame(root)
    root.mainloop()
